# Final Solution (First Try)

In [7]:
import pandas as pd
from pyomo.environ import ConcreteModel, Var, NonNegativeReals, Binary, Objective, Constraint, Reals, maximize
import math
import numpy as np
from pyomo.environ import SolverFactory

# ======================
# 数据读取与处理
# ======================
data = pd.read_excel("final_merged_data_with_predictions&loss.xlsx")

# 数据列说明（根据题中给出）:
# 企业代号: '企业代号'
# 优化融合模型违约率: '优化融合模型违约率' -> p_i
# 最高贷款额度: '最高贷款额度' -> L_max_i
# 流失损失: '流失损失' -> V_s_i
# 信誉评级数值: '信誉评级数值' -> R_i (假设A=1, B=2, C=3, D=4)
# 总额度：T=10000（万元)
beta = 0.22
alpha = 4.0
T = 10000
D_rating_value = 4

# 提取参数
N = data.shape[0]
p = data['优化融合模型违约率'].values
L_max_base = data['最高贷款额度'].values
V_s = data['流失损失'].values
R_values = data['信誉评级数值'].values
enterprise_ids = data['企业代号'].values

# 定义S(R_i,r_i)函数
def S(R, r):
    # 根据给定的多项式:
    # A级(R=1): 37.97r^3 - 258.57r^2 + 640.94r - 1.12
    # B级(R=2): 33.99r^3 - 225.05r^2 + 552.83r - 1.02
    # C级(R=3): 32.16r^3 - 207.39r^2 + 504.72r - 0.97
    # D级(R=4): 不放贷, 可设置S=1以确保在目标中不选择放贷
    
    val = 0  # Default value
    if R == 1:  # A
        val = 37.97*(r**3) - 258.57*(r**2) + 640.94*r - 1.12
    elif R == 2: # B
        val = 33.99*(r**3) - 225.05*(r**2) + 552.83*r - 1.02
    elif R == 3: # C
        val = 32.16*(r**3) - 207.39*(r**2) + 504.72*r - 0.97
    elif R == 4: # D
        # D级不放贷，设为1
        val = 1.0
    # 保证S在[0,1]
    #val = min(max(val, 0.0), 1.0)
    return val

# ======================
# 创建模型
# ======================
model = ConcreteModel()

# 定义变量
model.x = Var(range(N), within=Binary)   # 是否放贷决策变量
model.L = Var(range(N), within=NonNegativeReals)
model.r = Var(range(N), bounds=(0.04, 0.15))

# 信誉为D时不放贷
def no_loan_for_D_rule(m, i):
    if R_values[i] == D_rating_value:
        return m.L[i] == 0
    return Constraint.Skip
model.no_loan_for_D = Constraint(range(N), rule=no_loan_for_D_rule)

def no_loan_x_for_D_rule(m, i):
    if R_values[i] == D_rating_value:
        return m.x[i] == 0
    return Constraint.Skip
model.no_loan_x_for_D = Constraint(range(N), rule=no_loan_x_for_D_rule)

# 贷款额度上限约束
def loan_upper_rule(m, i):
    if R_values[i] != D_rating_value:
        if p[i] <= 0.4:
            L_max_adjusted = L_max_base[i]*math.exp(-alpha*p[i])
        else:
            # p_i >0.4时不建议放贷，给上限为0
            L_max_adjusted = 0
        return m.L[i] <= L_max_adjusted*m.x[i]
    else:
        return Constraint.Skip
model.loan_upper = Constraint(range(N), rule=loan_upper_rule)

# 贷款额度下限约束(若放贷则≥10万)
def loan_lower_rule(m, i):
    if R_values[i] != D_rating_value:
        return m.L[i] >= 10*m.x[i]
    else:
        return Constraint.Skip
model.loan_lower = Constraint(range(N), rule=loan_lower_rule)

# 总贷款额度约束
def total_loan_rule(m):
    return sum(m.L[i] for i in range(N)) <= T
model.total_loan = Constraint(rule=total_loan_rule)

# 目标函数
def obj_rule(m):
    expr = 0
    for i in range(N):
        R_i = R_values[i]
        p_i = p[i]
        V_s_i = V_s[i]
        r_i = m.r[i]
        L_i = m.L[i]
        s_i = S(R_i, r_i)  # 使用新定义的S函数
        expr += ( (L_i*(r_i - beta*p_i))*(1 - s_i) - s_i*V_s_i )
    return expr

model.obj = Objective(rule=obj_rule, sense=maximize)

# ==============
# 求解
# ==============
# 选择合适的MINLP求解器，如couenne或baron
solver = SolverFactory('couenne') # 也可以选择其他求解器
results = solver.solve(model, tee=True)

# ======================
# 获取结果并导出
# ======================
L_opt = [model.L[i].value for i in range(N)]
r_opt = [model.r[i].value for i in range(N)]
x_opt = [int(round(model.x[i].value)) for i in range(N)]  # 转为int 0/1
total_loan_amt = sum(L_opt)
ratio = [l/total_loan_amt if total_loan_amt>0 else 0 for l in L_opt]

result_df = pd.DataFrame({
    '企业代号': enterprise_ids,
    '是否放贷': x_opt,
    '放贷金额(万元)': L_opt,
    '放贷金额占比': ratio,
    '放贷利率': r_opt
})

result_df.to_excel("loan_decision_result.xlsx", index=False)




Couenne 0.5.6 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -119.44747       42 0.17
Couenne: new cutoff value -1.1944735237e+002 (0.182 seconds)
Loaded instance "C:\Users\Jerry\AppData\Local\Temp\tmp4dk68ms1.pyomo.nl"
Constraints:          199
Variables:            297 (99 integer)
Auxiliaries:          427 (0 integer)

Coin0506I Presolve 983 (-182) rows, 508 (-216) columns and 2727 (-278) elements
Clp0006I 0  Obj -119.43735 Primal inf 42.23025 (44) Dual inf 321.53442 (1)
Clp0006I 94  Obj -192.94329 Primal inf 418.89995 (39)
Clp0006I 188  Obj -192.94329 Primal inf 425.64282 (37)
Clp0006I 282  Obj -192.94329 Primal inf 40.235547 (36)
Clp0006I 359  Obj -119.44735
Clp0000I Optimal - objective value -119.44735
Clp0032I Optimal ob

In [2]:
# ======================
# 数据读取与处理
# ======================
import pandas as pd
from pyomo.environ import ConcreteModel, Var, NonNegativeReals, Binary, Objective, Constraint, maximize
import math
import numpy as np
from pyomo.environ import SolverFactory

# ======================
# 数据读取与处理
# ======================
data = pd.read_excel("final_merged_data_with_predictions&loss.xlsx")

beta = 0.22
alpha = 4.0
T = 10000
D_rating_value = 4

# 提取参数
N = data.shape[0]
p = data['优化融合模型违约率'].values
L_max_base = data['最高贷款额度'].values
V_s = data['流失损失'].values
R_values = data['信誉评级数值'].values
enterprise_ids = data['企业代号'].values

# 定义S(R_i, r_i)函数
def S(R, r):
    val = 0  # 默认值
    if R == 1:  # A级
        val = 37.97*(r**3) - 258.57*(r**2) + 640.94*r - 1.12
    elif R == 2: # B级
        val = 33.99*(r**3) - 225.05*(r**2) + 552.83*r - 1.02
    elif R == 3: # C级
        val = 32.16*(r**3) - 207.39*(r**2) + 504.72*r - 0.97
    elif R == 4: # D级
        val = 1.0  # D级直接返回1
    return val

# ======================
# 创建模型
# ======================
model = ConcreteModel()

# 定义变量
model.x = Var(range(N), within=Binary)   # 是否放贷决策变量
model.L = Var(range(N), within=NonNegativeReals)
model.r = Var(range(N), bounds=(0.04, 0.15))

# 信誉为D时不放贷
def no_loan_for_D_rule(m, i):
    if R_values[i] == D_rating_value:
        return m.L[i] == 0
    return Constraint.Skip
model.no_loan_for_D = Constraint(range(N), rule=no_loan_for_D_rule)

def no_loan_x_for_D_rule(m, i):
    if R_values[i] == D_rating_value:
        return m.x[i] == 0
    return Constraint.Skip
model.no_loan_x_for_D = Constraint(range(N), rule=no_loan_x_for_D_rule)

# 贷款额度上限约束
def loan_upper_rule(m, i):
    if R_values[i] != D_rating_value:
        if p[i] <= 0.4:
            L_max_adjusted = L_max_base[i]*math.exp(-alpha*p[i])
        else:
            L_max_adjusted = 0
        return m.L[i] <= L_max_adjusted*m.x[i]
    else:
        return Constraint.Skip
model.loan_upper = Constraint(range(N), rule=loan_upper_rule)

# 贷款额度下限约束(若放贷则≥10万)
def loan_lower_rule(m, i):
    if R_values[i] != D_rating_value:
        return m.L[i] >= 10*m.x[i]
    else:
        return Constraint.Skip
model.loan_lower = Constraint(range(N), rule=loan_lower_rule)

# 总贷款额度约束
def total_loan_rule(m):
    return sum(m.L[i] for i in range(N)) <= T
model.total_loan = Constraint(rule=total_loan_rule)

# 目标函数
def obj_rule(m):
    expr = 0
    for i in range(N):
        R_i = R_values[i]
        p_i = p[i]
        V_s_i = V_s[i]
        r_i = m.r[i]
        L_i = m.L[i]
        
        # 计算S(R_i, r_i)
        s_i = S(R_i, r_i)
        
        # 对于D级企业，直接跳过
        if R_i == D_rating_value:
            continue
        
        # 计算目标函数表达式
        expr += ( (L_i * (r_i - beta * p_i)) * (1 - s_i) - s_i * V_s_i )
    
    return expr

model.obj = Objective(rule=obj_rule, sense=maximize)

# ==============
# 求解
# ==============
solver = SolverFactory('couenne') # 或 'baron' 或其他
results = solver.solve(model, tee=True)

# ======================
# 获取结果并导出
# ======================
L_opt = [model.L[i].value for i in range(N)]
r_opt = [model.r[i].value for i in range(N)]
x_opt = [int(round(model.x[i].value)) for i in range(N)]  # 转为int 0/1
total_loan_amt = sum(L_opt)
ratio = [l/total_loan_amt if total_loan_amt>0 else 0 for l in L_opt]

result_df = pd.DataFrame({
    '企业代号': enterprise_ids,
    '是否放贷': x_opt,
    '放贷金额(万元)': L_opt,
    '放贷金额占比': ratio,
    '放贷利率': r_opt
})

result_df.to_excel("loan_decision_result.xlsx", index=False)



Couenne 0.5.6 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -119.44747       42 0.169
Couenne: new cutoff value -1.1944735237e+002 (0.195 seconds)
Loaded instance "C:\Users\Jerry\AppData\Local\Temp\tmpn69d4iuu.pyomo.nl"
Constraints:          199
Variables:            297 (99 integer)
Auxiliaries:          427 (0 integer)

Coin0506I Presolve 983 (-182) rows, 508 (-216) columns and 2727 (-278) elements
Clp0006I 0  Obj -119.43735 Primal inf 42.23025 (44) Dual inf 321.53442 (1)
Clp0006I 94  Obj -192.94329 Primal inf 418.89995 (39)
Clp0006I 188  Obj -192.94329 Primal inf 425.64282 (37)
Clp0006I 282  Obj -192.94329 Primal inf 40.235547 (36)
Clp0006I 359  Obj -119.44735
Clp0000I Optimal - objective value -119.44735
Clp0032I Optimal o

In [16]:
import pandas as pd
import numpy as np

# 读取已有的loan_decision_result.xlsx文件
result_df = pd.read_excel("loan_decision_result.xlsx")

# 设定最低和最高利率
min_rate = 0.04
max_rate = 0.15

# 假设放贷金额占比（'放贷金额占比'）越高，利率应越低
def generate_loan_rate(p):
    """
    根据放贷金额占比计算贷款利率。
    p: 放贷金额占比 (0到1之间)
    """
    if p == 0:
        return 0  # 不放贷时，利率为0
    
    # 反向映射：占比越高，利率越低
    rate = max_rate - (max_rate - min_rate) * p**0.5  # 线性反向映射
    
    # 保证利率在[min_rate, max_rate]区间
    rate = np.clip(rate, min_rate, max_rate)
    
    return rate

# 遍历每个企业，基于是否放贷以及其放贷金额占比，计算放贷利率
for i in range(len(result_df)):
    if result_df['是否放贷'][i] == 1:  # 如果放贷
        p = result_df['放贷金额占比'][i]  # 使用放贷金额占比
        result_df.at[i, '放贷利率'] = generate_loan_rate(p)  # 计算新的利率
    else:  # 不放贷的企业，放贷利率设为0
        result_df.at[i, '放贷利率'] = 0

# 保存修改后的DataFrame为新的Excel文件
result_df.to_excel("updated_loan_decision_result.xlsx", index=False)

print("修改后的贷款利率已保存为 'updated_loan_decision_result.xlsx'")



修改后的贷款利率已保存为 'updated_loan_decision_result.xlsx'


In [18]:
import pandas as pd
import numpy as np

# 读取已有的loan_decision_result.xlsx文件
result_df = pd.read_excel("loan_decision_result.xlsx")

# 设定最低和最高利率
min_rate = 0.04
max_rate = 0.15

# 假设放贷金额占比（'放贷金额占比'）越高，利率应越低
def generate_loan_rate(p):
    """
    根据放贷金额占比计算贷款利率。
    p: 放贷金额占比 (0到1之间)
    """
    if p == 0:
        return 0  # 不放贷时，利率为0
    
    # 反向映射：占比越高，利率越低
    rate = max_rate - (max_rate - min_rate) * p**0.5  # 线性反向映射
    
    # 保证利率在[min_rate, max_rate]区间
    rate = np.clip(rate, min_rate, max_rate)
    
    return rate

# 加入正态分布的随机波动
def add_random_fluctuation(rate):
    """
    在给定的贷款利率基础上加入波动。
    利用正态分布生成波动，范围在[-0.05, 0.05]之间，波动较小的概率较大。
    """
    # 生成正态分布波动（均值0，标准差0.05）
    fluctuation = np.random.normal(0, 0.03)
    
    # 新的利率 = 原利率 + 波动
    new_rate = rate + fluctuation
    
    # 保证利率在[min_rate, max_rate]区间
    new_rate = np.clip(new_rate, min_rate, max_rate)
    
    return new_rate

# 遍历每个企业，基于是否放贷以及其放贷金额占比，计算放贷利率
for i in range(len(result_df)):
    if result_df['是否放贷'][i] == 1:  # 如果放贷
        p = result_df['放贷金额占比'][i]  # 使用放贷金额占比
        base_rate = generate_loan_rate(p)  # 计算基础利率
        result_df.at[i, '放贷利率'] = add_random_fluctuation(base_rate)  # 加入随机波动
    else:  # 不放贷的企业，放贷利率设为0
        result_df.at[i, '放贷利率'] = 0

# 保存修改后的DataFrame为新的Excel文件
result_df.to_excel("updated_loan_decision_result_with_fluctuations.xlsx", index=False)

print("修改后的贷款利率已保存为 'updated_loan_decision_result_with_fluctuations.xlsx'")


修改后的贷款利率已保存为 'updated_loan_decision_result_with_fluctuations.xlsx'
